In [6]:
from numpy.random import Generator
import numpy as np

In [24]:
import argparse
import os
from time import time

import evaluate
import lightning as pl
import numpy as np
import pandas as pd
import PIL
import torch
import torchvision
import transformers
from colorama import Back, Fore, Style
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import Linear, Sequential
from torch.nn import functional as TF
from torch.optim import SGD, AdamW, lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchmetrics.classification import F1Score, MulticlassAccuracy
from transformers import (
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from pathlib import Path

In [16]:

def get_data(train_size=5000):
    """
    Create a subset of data
    """

    def preprocess_function(examples):
        return tokenizer_obj(examples["text"], truncation=True)
    
    rng = np.random.default_rng(12345)
    val_size = train_size // 10
    test_size = train_size // 10

    dataset_dict = load_dataset("imdb")
    
    idx = rng.choice(
        np.arange(len(dataset_dict["train"])), train_size + val_size, replace=False
    )
    idx_train = idx[:train_size]
    idx_val = idx[train_size:]

    train_dataset = dataset_dict["train"].select(idx_train)
    val_dataset = dataset_dict["train"].select(idx_val)
    
    idx = rng.choice(
        np.arange(len(dataset_dict["test"])), test_size, replace=False
    )
    dataset_dict["test"] = dataset_dict["test"].select(idx)
    dataset_dict = DatasetDict(
        {
            "train": train_dataset,
            "validation": val_dataset,
            "test": dataset_dict["test"],
        }
    )

    return dataset_dict

## Description of dataset and splits

In [35]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

In [34]:
data = get_data()

## Data samples

In [38]:
data['train'][10]

{'text': "The idea was awesome, the actors were incredible, the story could of been very scary, but the writing was poor and there was no depth. I couldn't really get into this movie. I couldn't feel for the characters, there were a lot of cliffhangers, and the movie just ends very weirdly. Was it a happy ending? I don't know. Was it a sad ending? Again, I don't know. You leave the theater feeling unsatisfied. The movie had so much to give, but couldn't. Just because you can edit, doesn't mean you should, right? I wouldn't really recommend this movie because you just can't say that you left the movie feeling like it was completed. You'll just be confused. Trust me, you will probably thank me if you don't watch this movie.<br /><br />3/10",
 'label': 0}

In [39]:
data['train'][100]

{'text': 'Its time to pay tribute to the great Charton Heston after his recent passing but this film is not the one. His other films of a past generation were BEN HUR, THE TEN COMMANDENTS, OMEGA MAN and PLANET OF THE APES were his better works.<br /><br />This film made in 1973 attempts to prophesies a future earth , in 2022, that is so overpopulated that the human race has been manipulated by authorities to eat a universally produced food product called "Soylent Green" which is manufactured with Human flesh. This bizarre and implausible film was as ridiculous at the time of its release as it is now and assumes India\'s population which would be about 2 billion by that stage would be then meat eaters without knowing it.<br /><br />Charlton Heston\'s character this supers secret international conspiracy that world powers have concocted to meet the nutritional demands of overpopulation by using cannibalism.<br /><br />Unfortunately for the producers of this film the Green message they de